In [207]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1530808215043_0012,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1530808215043_0012,spark,idle,Link,Link,✔


In [210]:
// The Kafka broker hosts and topic used to write to Kafka
val kafkaBrokers="wn0-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:9092,wn1-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:9092"
val kafkaTopic="rawevents1"
val zookeepers = "zk4-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181,zk5-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181"

zookeepers: String = zk4-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181,zk5-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181

In [463]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 3 --partitions 8 --topic streamPerHour --zookeeper "zk4-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181,zk5-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181"

Created topic "streamPerHour".


In [409]:
// Import bits useed for declaring schemas and working with JSON data
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

// Define a schema for the data
val schema = (new StructType).
add("house_id", StringType).
add("timestamp", StringType).
add("value", StringType).
add("household_id", StringType)

schema: org.apache.spark.sql.types.StructType = StructType(StructField(house_id,StringType,true), StructField(timestamp,StringType,true), StructField(value,StringType,true), StructField(household_id,StringType,true))

In [212]:
import org.apache.spark.sql.streaming.Trigger;

val console_op_df = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", kafkaTopic).
option("startingOffsets", "earliest").
load()
                                 
var console_op_query = console_op_df.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)").as[(String,String)].
select(from_json(col("value"), schema) as "consumption").
writeStream.
format("console").
outputMode("append").
start()


console_op_query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@755cd645

In [284]:
console_op_query.stop()

In [457]:
val kafka_op_df = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", kafkaTopic).
option("startingOffsets","latest").
load()
                                 
var kafka_op_query = kafka_op_df.selectExpr("CAST(key AS STRING) AS key","to_json(struct(*)) AS value").as[(String,String)].
writeStream.
format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("checkpointLocation", "/path/checkpointloc").
option("topic", "cleanseevents").
option("failOnDataLoss","false").
outputMode("append").
start()

kafka_op_query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6cc16343

In [459]:
kafka_op_query.status

res135: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}

In [327]:
val csv_op_df = spark.readStream.format("kafka").
option("kafka.bootstrap.servers", kafkaBrokers).
option("subscribe", kafkaTopic).
option("startingOffsets","latest").
load()

var csv_op_query = csv_op_df.selectExpr("CAST(key AS STRING) AS Key","CAST(value AS STRING) AS value").as[(String,String)].
select(from_json(col("value"), schema).alias("consumption")).
select("consumption.*").
select(col("house_id").cast(IntegerType)).
writeStream.
format("csv").
option("path","/example/csvdata").
option("checkpointLocation", "/path/chkpoint1").
outputMode("append").
start()                                 

csv_op_query: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@662e98e9

In [447]:
csv_op_query.status


res126: org.apache.spark.sql.streaming.StreamingQueryStatus =
{
  "message" : "Processing new data",
  "isDataAvailable" : true,
  "isTriggerActive" : true
}

In [456]:
csv_op_query.stop()

In [448]:
%%bash
hdfs dfs -ls /example/csvdata

Found 13 items
drwxr-xr-x   - livy supergroup          0 2018-07-06 08:41 /example/csvdata/_spark_metadata
-rw-r--r--   1 livy supergroup          0 2018-07-06 08:41 /example/csvdata/part-00000-32a66c99-adda-4a03-86c0-27adf4431205-c000.csv
-rw-r--r--   1 livy supergroup         10 2018-07-06 08:41 /example/csvdata/part-00000-9af34090-e208-44bd-b01f-ab56a549da2c-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 08:41 /example/csvdata/part-00001-56c475ac-012e-469a-be98-cdef914da903-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 08:41 /example/csvdata/part-00002-fc2d0338-85d5-4775-b3a7-f9a128f8a612-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 08:41 /example/csvdata/part-00003-078e2bd7-0dc4-4d8e-8850-db30375c1edf-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 08:41 /example/csvdata/part-00003-722d7334-fb92-4285-907e-0b66cbe6bef6-c000.csv
-rw-r--r--   1 livy supergroup          0 2018-07-06 08:41 /example/csvdata/part-00004-b774ddff

In [454]:
%%bash
hdfs dfs -rm -r /path/checkpointloc

18/07/06 08:45:42 WARN azure.AzureFileSystemThreadPoolExecutor: Disabling threads for Rename operation as thread count 0 is <= 1
18/07/06 08:45:44 INFO azure.AzureFileSystemThreadPoolExecutor: Time taken for Rename operation is: 1844 ms with threads: 0
18/07/06 08:45:44 INFO fs.TrashPolicyDefault: Moved: 'wasb://sidsparkcluster@storesidkafkacluster.blob.core.windows.net/path/checkpointloc' to trash at: wasb://sidsparkcluster@storesidkafkacluster.blob.core.windows.net/user/spark/.Trash/Current/path/checkpointloc


In [449]:
%%bash
hdfs dfs -cat /example/csvdata/part-00000-9af34090-e208-44bd-b01f-ab56a549da2c-c000.csv

0
0
0
0
0


In [360]:
%%bash
hdfs dfs -cat /example/jsondata/part-00000-56912788-fe73-401d-8a5b-3c73b3128864-c000.csv

0,"{\"house_id\": \"0\", \"timestamp\": \"1378272960\", \"value\": \"2533.609000000001\", \"household_id\": \"0\"}"
0,"{\"house_id\": \"0\", \"timestamp\": \"1378273020\", \"value\": \"2515.039000000001\", \"household_id\": \"0\"}"
0,"{\"house_id\": \"0\", \"timestamp\": \"1378273080\", \"value\": \"2454.283000000002\", \"household_id\": \"0\"}"
0,"{\"house_id\": \"0\", \"timestamp\": \"1378273140\", \"value\": \"2552.0789999999984\", \"household_id\": \"0\"}"
0,"{\"house_id\": \"0\", \"timestamp\": \"1378273200\", \"value\": \"2534.137000000001\", \"household_id\": \"0\"}"
0,"{\"house_id\": \"0\", \"timestamp\": \"1378273260\", \"value\": \"2483.291\", \"household_id\": \"0\"}"


In [228]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --list --zookeeper "zk4-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181,zk5-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181"


__consumer_offsets
cleanseevents
rawevents1


In [108]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-consumer-groups.sh  --bootstrap-server "wn0-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:9092,wn1-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:9092" --list 

console-consumer-62414
console-consumer-2211
spark-kafka-source-69018759-e09a-4381-b790-8cc7c3d01e8b--673031480-driver-0


In [107]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-consumer-groups.sh  --bootstrap-server "wn0-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:9092,wn1-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:9092" --describe  --group  "spark-kafka-source-2020e267-2a5f-4b4a-89f2-f4f009a2a703--1041004857-executor"

Consumer group `spark-kafka-source-2020e267-2a5f-4b4a-89f2-f4f009a2a703--1041004857-executor` does not exist.


In [460]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --delete --topic "rawevents1" --zookeeper "zk4-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181,zk5-kafka.y4m1vjdw55he5ghlbslra2ouva.cx.internal.cloudapp.net:2181"


Topic rawevents1 is marked for deletion.
Note: This will have no impact if delete.topic.enable is not set to true.
